In [2]:
# 챔피언 리스트를 웹사이트로부터 가져온다.
del_list = ["아우렐리온 솔", "미스 포츈","리 신", "마스터 이", "문도 박사", "다리우스", "리산드라", "말자하",
            "신 짜오","자르반 4세", "트위스티드 페이트","누누와 윌럼프","아트록스","오리아나", "말파이트", "모데카이저",
           "블라디미르", "세주아니", "아펠리오스", "알리스타", "이렐리아", "이즈리얼","카시오페아", "카타리나",
           "칼리스타", "킨드레드", "트린다미어", "피들스틱", "하이머딩거", "갱플랭크", "탐 켄치"]
# del_list는 실제 풀 네임으로 부르지 않는 경우나 줄임말로도 포함이 가능한 경우를 고려해 삭제하려는 리스트
another = ["갱플", "그브", "누누", "다리", "리산", '마이', '말자', '말파', '모데', '문도', '미포', '볼베', 
'블라디', '블츠', '세주', '짜오', '아우솔', '아펠', '알리', '오리', '이렐', '이즈', '자르반', '카시', 
'카타', '칼리', '케틀', '킨드', '트타', '트린', '트페', '빵테', '피들', '딩거', '리신', '아트', '켄치']
# another_list는 챔피언 명을 줄여서 부르는 경우를 정리하여 추가하려는 리스트
html = requests.get("https://lol.inven.co.kr/dataninfo/champion/").text
soup = BeautifulSoup(html,"lxml")
res = soup.find_all("div" , {"class":"champName"})
# python 에서 url 처리 시 한글이 포함된 경우 깨짐현상이 발생하는걸 방지하기 위해 quote 처리
champions_quote = []
champions = []


for one in res:
    champions.append(one.get_text())
for i in del_list:
    champions.remove(i)
for a in another:
    champions.append(a)
for one in res:
    champions_quote.append(quote(one.get_text()))
for k in del_list:
    champions_quote.remove(quote(k))
for a in another:
    champions_quote.append(quote(a))
len(champions_quote)

163

### 가져온 챔피언 리스트를 이용하여 웹크롤링

In [4]:
# 가져온 챔피언 리스트를 크롤링하고자 하는 웹 url에 대응시켜 반복한다.
headers = {
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
    }
result= []
# for champname_quote in ['%EA%B0%80%EB%A0%8C']: (테스트용 라인)
for champname_quote,champ in zip(champions_quote[:41], champions[:41]):
# 전체 리스트를 서치할 경우 트래픽에 의한 연결 해제 문제가 발생, 리스트를 나눠서 검색
    url = f"https://gall.dcinside.com/board/lists?id=leagueoflegends4&s_type=search_subject&s_keyword={champname_quote}"
    url_list= []

    for _ in range(200):

    #     print("url:", url)
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")
        div_res = soup.find("div", {"class":"bottom_paging_box"})

        a_next_res = div_res.find("a", {"class":"search_next"})
        em_res = div_res.find("em")

#             if a_next_res == None:
#                 print(unquote(champname_quote), "찾기 끝")
#                 break

        cur_page = "0"
        if em_res != None:
            cur_page = em_res.get_text()
    #         print("page",cur_page)

        fix_url = "https://gall.dcinside.com"
        if cur_page =="1":
            a_prev_res = div_res.find("a", {"class":"search_prev"})
            a_res = div_res.find_all("a")
            if a_prev_res: a_res.remove(a_prev_res)

            for one in a_res:
                url_list.append(fix_url+one["href"])
        elif cur_page =="0":
    #         print("이 구간에 조회 내용 없음")
            url_list.append(fix_url+a_next_res["href"])

        url = url_list.pop(0)

        table = soup.find("tbody")
        trs = table.find_all("tr")
        for sd in trs:
            tds = sd.find_all("td")   
            title = tds[1].get_text()
            title = title.replace("\n","")
#             스크래핑 당시 게시판에 랜덤하게 발생하는 광고글을 제외하기 위한 구문
            if title == "[와쿠] 게임 추천하고 케이크 받자!" or title == "알고 보니 재력가 집안 스타는?" or title == "4차산업혁명, 나도인재가 될수있다! 3D프린터/언리얼게임 무료교육 진행":
                continue
            date  = tds[3].get_text()
            if len(date) == 5 and date.find(".")>=0:
                date = date.replace(".", "")
                date = "21"+date
            elif len(date) == 8 and date.find(".")>=0:
                date = date.replace(".", "")
            elif len(date) == 5 and date.find(":")>=0:
                date = "211231" # 스크래핑한 날짜

#             print(champ, "/", title, "/", date)
            result.append([champ]+[title]+[date])
        time.sleep(0.1)
    print("한 챔프 끝")

col_name = ["챔피언","제목","날짜"]
df_dc = pd.DataFrame(result, columns = col_name)
df_dc.to_csv("dc_data41.csv", encoding="utf-8")

한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝
한 챔프 끝


In [5]:
df_dc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43587 entries, 0 to 43586
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   챔피언     43587 non-null  object
 1   제목      43587 non-null  object
 2   날짜      43587 non-null  object
dtypes: object(3)
memory usage: 1021.7+ KB
